In [1]:
import sys
import os
import json
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from overpass_client import OverpassClient
city = "Kraków"


### Download roads and tram stops using overpass client

In [2]:
result = OverpassClient.get_tram_stops_and_crossroads(city)

ways_data = []
for way in result.ways:
    ways_data.append({
        "id": way.id,
        "nodes": way._node_ids,
        "tags": way.tags,  
    })

tram_stops = [
    node for node in result.nodes
    if node.tags.get("railway") == "tram_stop"
]

tram_stops_data = [
    {"id": node.id, "lat": float(node.lat), "lon": float(node.lon), "name": node.tags.get("name")}
    for node in tram_stops
]


with open("tram_ways.json", "w", encoding="utf-8") as f:
    json.dump(ways_data, f, indent=4, ensure_ascii=False)

with open("tram_stops.json", "w", encoding="utf-8") as f:
    json.dump(tram_stops_data, f, indent=4, ensure_ascii=False)

In [3]:
result = OverpassClient.get_tram_stops_and_crossroads_coordinate(city)
nodes_data = []
for node in result.nodes:
    nodes_data.append({
        "id": node.id,
        "lat": float(node.lat),
        "lon": float(node.lon),
    })
with open("tram_stops_crossroads_coords.json", "w", encoding="utf-8") as f:
    json.dump(nodes_data, f, indent=4, ensure_ascii=False)

result = OverpassClient.get_track_geometry(city)
features = []

for way in result.ways:
    coords = [(float(node.lon), float(node.lat)) for node in way.nodes]
    features.append({
        "type": "Feature",
        "properties": {
            "id": way.id,
            "tags": way.tags
        },
        "geometry": {
            "type": "LineString",
            "coordinates": coords
        }
    })

geojson = {
    "type": "FeatureCollection",
    "features": features
}

with open("track_geometry.json", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)